In [1]:
%%capture
!pip install transformers
import torch
import torch
from transformers import BertTokenizer, BertModel

In [2]:
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

In [4]:
text='私はこの作品を3日間作ろうとしています'

In [5]:
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
print(len(tokenized_text))

['私', 'は', '##こ', '##の', '作', '品', 'を', '##3', '日', '間', '作', 'ろう', '##と', '##して', '##い', '##ます']
16


In [6]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])

In [7]:
tokenizer(text)

{'input_ids': [2, 3946, 897, 6550, 6462, 1159, 1619, 932, 6221, 2719, 5489, 1159, 19431, 6461, 11513, 6323, 20114, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-v2')

In [9]:
with torch.no_grad():
    print(tokens_tensor.shape)
    outputs = model.forward(input_ids=tokens_tensor,return_dict=True)

torch.Size([1, 16])


In [10]:
print(outputs['pooler_output'].shape) #use this for classification

torch.Size([1, 768])


In [2]:


# Tokenize input
text = "私はこの作品を3日間作ろうとしています"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

INFO:filelock:Lock 139991696330896 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


INFO:filelock:Lock 139991696330896 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
INFO:filelock:Lock 139991696834320 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


INFO:filelock:Lock 139991696834320 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
INFO:filelock:Lock 139991699769488 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


INFO:filelock:Lock 139991699769488 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
source_folder = '/content/drive/MyDrive/data_rakuten' #source folder of csv files

In [ ]:
#Load data

XTrain = pd.read_csv(os.path.join(source_folder,'X_train_12tkObq.csv'), index_col=0)
XTest = pd.read_csv(os.path.join(source_folder,'X_test_gDTIJPh.csv'), index_col=0)

YTrain = pd.read_csv(os.path.join(source_folder,'y_train_Q9n2dCu.csv'), index_col=0)
YTrain['color_tags'] = YTrain['color_tags'].apply(lambda x: ast.literal_eval(x)) #to change str to list labels

In [ ]:
class TextDataset(torch.utils.data.Dataset):

    def __init__(self,features,labels):
      #Load pre-computed tensors
      self.features=features
      self.labels=labels
        
    def __len__(self):
        return self.features.shape[1]

    def __getitem__(self, idx):

        return  self.features[:,idx],self.labels[:,idx]

trainSet= TextDataset(X_features[:,:int(X_features.shape[1]*ProportionTrainVal)],Ytrain_label[:,:int(X_features.shape[1]*ProportionTrainVal)])
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=64,shuffle=True, num_workers=2)

valSet= TextDataset(X_features[:,int(X_features.shape[1]*ProportionTrainVal):],Ytrain_label[:,int(X_features.shape[1]*ProportionTrainVal):])
valLoader = torch.utils.data.DataLoader(valSet, batch_size=64,shuffle=False, num_workers=2)

In [ ]:
class CustomModel(torch.nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()

        self.tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
        self.encoder= BertModel.from_pretrained('cl-tohoku/bert-base-japanese')
        for param in encoder.parameters():  #Freeze the weights of BERT because we want to use it only for sentence embedding
            param.requires_grad = False


    def forward(self, text_features):
        text_features = F.relu(self.fc1(text_features))
        text_features = F.relu(self.fc2(text_features))
        #text_features = self.dropout(text_features)
        logits = self.fc3(text_features)

        return logits

model=CustomModel()
model.to(device)